In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from pandas import json_normalize
import re

In [ ]:


pd.set_option("max_colwidth", 180)
data = pd.read_csv("movies_dataset.csv")



# ETL


Eliminamos las columnas que no se utilizarán: video, imdb_id, adult, original_title, poster_pathy homepage.

In [52]:
data.drop(['adult','belongs_to_collection','status', 'video','imdb_id', 'original_title', 'poster_path', 'homepage'], axis=1,inplace=True)

Eliminamos los duplicados y algunos registros que detectamos con errores 

In [53]:
data = data.drop_duplicates()
data = data.drop(index=35587)
data = data.drop(index=19730)
data = data.drop(index=29503)

data = data.dropna(subset=['title'])


In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45447 entries, 0 to 45465
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45447 non-null  object 
 1   genres                45447 non-null  object 
 2   id                    45447 non-null  object 
 3   original_language     45436 non-null  object 
 4   overview              44493 non-null  object 
 5   popularity            45447 non-null  object 
 6   production_companies  45447 non-null  object 
 7   production_countries  45447 non-null  object 
 8   release_date          45363 non-null  object 
 9   revenue               45447 non-null  float64
 10  runtime               45190 non-null  float64
 11  spoken_languages      45447 non-null  object 
 12  tagline               20408 non-null  object 
 13  title                 45447 non-null  object 
 14  vote_average          45447 non-null  float64
 15  vote_count         

Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados y tienen datos faltantes

In [55]:

data['genres']=data['genres'].apply(lambda x: {} if pd.isna(x) else x)
data['production_companies']=data['production_companies'].apply(lambda x: {} if pd.isna(x) else x)
data['production_countries']=data['production_countries'].apply(lambda x: {} if pd.isna(x) else x)
data['spoken_languages']=data['spoken_languages'].apply(lambda x: {} if pd.isna(x) else x)


Cambiamos comillas simples por dobles en estas columnas ya que pandas suele dar error 

In [57]:

data['genres']={k: v.replace("'", '"') if isinstance(v, str) else v for k, v in data['genres'].items()}
data['production_companies']={k: v.replace("'", '"') if isinstance(v, str) else v for k, v in data['production_companies'].items()}
data['production_countries']={k: v.replace("'", '"') if isinstance(v, str) else v for k, v in data['production_countries'].items()}
data['spoken_languages']={k: v.replace("'", '"') if isinstance(v, str) else v for k, v in data['spoken_languages'].items()}

Cambiamos el tipo de datos de estas columnas 

In [58]:


data['spoken_languages'] = data['spoken_languages'].astype('string')
data['overview'] = data['overview'].astype('string')
data['tagline'] = data['tagline'].astype('string')
data['original_language'] = data['original_language'].astype('string')
data['title'] = data['title'].astype('string')
data['genres'] = data['genres'].astype('string')
data['popularity']=pd.to_numeric(data['popularity'])
data['budget']=pd.to_numeric(data['budget'])
data['id'] = data['id'].astype(int)
data['release_date']=pd.to_datetime(data['release_date'],errors='coerce')



In [59]:
def remove_genres(string):
    genres = ['[{\'', ':', 'iso_','iso__ **','_','name', 'id','\'}]', '\'', '1', '2','3','4','5','6','7','8','9','0','{','}','[',']',',','"']
    for genre in genres:
        if not pd.isna(string):
            string = string.replace(genre, '')
    return string


data['production_countries'] = data['production_countries'].apply(remove_genres)
data['production_companies'] = data['production_companies'].apply(remove_genres)
data['spoken_languages'] = data['spoken_languages'].apply(remove_genres)
data['genres'] = data['genres'].apply(remove_genres)

Cambiamos el tipo de datos de estas columnas desanidadas

In [60]:
data['production_companies'] = data['production_companies'].astype('string')
data['production_countries'] = data['production_countries'].astype('string')
data['spoken_languages'] = data['spoken_languages'].astype('string')
data['genres'] = data['genres'].astype('string')

Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0. Otros los rellenamos con 'Unavailable'

In [61]:
data['revenue']=data['revenue'].fillna(0)
data['budget']=data['budget'].fillna(0)
data['release_date']=data['release_date'].fillna(method='ffill')
data['tagline']=data['tagline'].fillna('*Unavailable')
data['runtime']=data['runtime'].fillna(0)
data['overview']=data['overview'].fillna('*Unavailable')
data['original_language']=data['original_language'].fillna('*Unavailable')


### Con los datos ya limpios procedemos a cumplir otros requerimientos del proyecto 

In [63]:

dataJ=data[['id','title', 'genres','overview','tagline','spoken_languages','original_language',
 'budget','revenue','popularity', 'release_date','runtime','production_companies','production_countries','vote_average', 'vote_count']]

deberá crear la columna release_year donde extraerán el año de la fecha de estreno.

In [64]:
dataJ['release_year'] = dataJ['release_date'].dt.year

C:\Users\dforc\AppData\Local\Temp\ipykernel_3332\525829048.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataJ['release_year'] = dataJ['release_date'].dt.year




Creamos la columna con el retorno de inversión, llame return con los campos revenuey budget, dividiendo estas dos últimas revenue / budget, cuando no haya datos disponibles para calcularlo, deberá tomar el valor 0.



In [ ]:
def calculate_return(row):
    if row['revenue'] == 0 or row['budget'] == 0:
        return 0
    else:
        return row['revenue'] / row['budget']

dataJ['return'] = dataJ.apply(calculate_return, axis=1)

### Hacemos la misma limpieza con el dataset **credits.csv**

In [67]:
credits = pd.read_csv("credits.csv")

In [68]:
credits = credits.drop_duplicates()
credits['cast'] = credits['cast'].astype('string')
credits['crew'] = credits['crew'].astype('string')

In [69]:
creditsW=credits[['id', 'cast','crew' ]]

In [70]:

def extract_name(df, column_name):
    def extract_name_from_string(string):
        data = ast.literal_eval(string)
        result = [d['name'] for d in data]
        return result

    df['c_crew'] = df[column_name].apply(extract_name_from_string)
    return df

creditsW = extract_name(creditsW, 'crew')
creditsW.drop(['crew'], axis=1,inplace=True)


C:\Users\dforc\AppData\Local\Temp\ipykernel_3332\1985400701.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['c_crew'] = df[column_name].apply(extract_name_from_string)


In [71]:
def extract_name(df, column_name):
    def extract_name_from_string(string):
        data = ast.literal_eval(string)
        result = [d['name'] for d in data]
        return result

    df['c_cast'] = df[column_name].apply(extract_name_from_string)
    return df

creditsW = extract_name(creditsW, 'cast')
creditsW.drop(['cast'], axis=1,inplace=True)

In [72]:
credits_ready=creditsW[['id', 'c_cast','c_crew']]
credits_ready=credits_ready.rename(columns={'c_cast': 'cast', 'c_crew': 'crew'})

In [73]:
credits_ready['cast'] = credits_ready['cast'].astype('string')
credits_ready['crew'] = credits_ready['crew'].astype('string')

### Finalmente unimos las columnas seleccionadas del dataset **movies_dataset.csv** con las columnas seleccionadas del dataset **credits.csv** para crear una sola base de datos

In [74]:
db= pd.merge(dataJ, credits_ready, on='id')

In [75]:
db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45459 entries, 0 to 45458
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    45459 non-null  int32         
 1   title                 45459 non-null  string        
 2   genres                45459 non-null  string        
 3   overview              45459 non-null  string        
 4   tagline               45459 non-null  string        
 5   spoken_languages      45459 non-null  string        
 6   original_language     45459 non-null  string        
 7   budget                45459 non-null  int64         
 8   revenue               45459 non-null  float64       
 9   popularity            45459 non-null  float64       
 10  release_date          45459 non-null  datetime64[ns]
 11  runtime               45459 non-null  float64       
 12  production_companies  45459 non-null  string        
 13  production_count

In [76]:

db.to_parquet('dbLista.parquet', index=False)